# Phase 2 Art Curation Integration Test (Async Fixed)

async/await 문제를 해결한 버전입니다.

In [1]:
import os
import sys
sys.path.append('/Users/ijunhyeong/Desktop/arti_llm/studio')

# 환경 변수 설정
from dotenv import load_dotenv
load_dotenv(override=True)

print("✅ 환경 설정 완료")

✅ 환경 설정 완료


In [2]:
# 그래프 로드 테스트
from emotional_art_graph import graph, EmotionalArtState
from langchain_core.messages import HumanMessage

print("✅ 그래프 로드 성공")
print(f"그래프 노드들: {list(graph.get_graph().nodes.keys())}")

/opt/anaconda3/envs/arti_chatbot/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/anaconda3/envs/arti_chatbot/lib/python3.11/site-packages/transformers/utils/hub.py:111: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


✅ 그래프 로드 성공
그래프 노드들: ['__start__', 'conversation_node', 'art_curation_node', '__end__']


In [3]:
# 그래프 구조 확인
graph_info = graph.get_graph()
print("=== 그래프 구조 ===")
print(f"노드들: {list(graph_info.nodes.keys())}")
print(f"엣지들: {graph_info.edges}")

# Mermaid 다이어그램 출력
try:
    mermaid_code = graph.get_graph().draw_mermaid()
    print("\n=== Mermaid 다이어그램 ===")
    print(mermaid_code)
except Exception as e:
    print(f"그래프 시각화 실패: {e}")

=== 그래프 구조 ===
노드들: ['__start__', 'conversation_node', 'art_curation_node', '__end__']
엣지들: [Edge(source='__start__', target='conversation_node', data=None, conditional=False), Edge(source='conversation_node', target='__end__', data=None, conditional=True), Edge(source='conversation_node', target='art_curation_node', data=None, conditional=True), Edge(source='art_curation_node', target='__end__', data=None, conditional=False)]

=== Mermaid 다이어그램 ===
---
config:
  flowchart:
    curve: linear
---
graph TD;
	__start__([<p>__start__</p>]):::first
	conversation_node(conversation_node)
	art_curation_node(art_curation_node)
	__end__([<p>__end__</p>]):::last
	__start__ --> conversation_node;
	conversation_node -.-> __end__;
	conversation_node -.-> art_curation_node;
	art_curation_node --> __end__;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill:#bfb6fc



In [4]:
# should_curate_art 함수 직접 테스트
from emotional_art_graph import should_curate_art

print("=== should_curate_art 함수 직접 테스트 ===")

test_cases = [
    {"current_phase": "greeting", "consent_for_reco": False},
    {"current_phase": "ready_for_curation", "consent_for_reco": False},
    {"current_phase": "deep_sensing", "consent_for_reco": True},
    {"current_phase": "ready_for_curation", "consent_for_reco": True},
]

for i, test_state in enumerate(test_cases):
    result = should_curate_art(test_state)
    print(f"테스트 {i+1}: {test_state} -> {result}")

=== should_curate_art 함수 직접 테스트 ===
DEBUG: Routing to END - phase: greeting, consent: False
테스트 1: {'current_phase': 'greeting', 'consent_for_reco': False} -> __end__
DEBUG: Routing to art curation - phase: ready_for_curation, consent: False
테스트 2: {'current_phase': 'ready_for_curation', 'consent_for_reco': False} -> art_curation_node
DEBUG: Routing to art curation - phase: deep_sensing, consent: True
테스트 3: {'current_phase': 'deep_sensing', 'consent_for_reco': True} -> art_curation_node
DEBUG: Routing to art curation - phase: ready_for_curation, consent: True
테스트 4: {'current_phase': 'ready_for_curation', 'consent_for_reco': True} -> art_curation_node


In [5]:
# 모듈 강제 reload
import importlib
import sys

if 'emotional_art_graph' in sys.modules:
    importlib.reload(sys.modules['emotional_art_graph'])

from emotional_art_graph import graph
print("✅ 그래프 다시 로드됨")

✅ 그래프 다시 로드됨


In [6]:
# 기본 대화 테스트 (ASYNC 사용)
config = {
    "configurable": {
        "thread_id": "test_user_123"
    }
}

initial_state = {
    "messages": [HumanMessage(content="Hello, I'm feeling stressed from work.")],
    "emotion_hints": [],
    "situation_hints": [],
    "current_phase": "greeting"
}

print("=== 기본 대화 테스트 (ASYNC) ===")
print("실행 전 상태:")
print(f"- current_phase: {initial_state.get('current_phase')}")
print(f"- consent_for_reco: {initial_state.get('consent_for_reco', False)}")

# should_curate_art로 라우팅 확인
routing_result = should_curate_art(initial_state)
print(f"예상 라우팅: {routing_result}")

try:
    # ASYNC 호출 사용
    result = await graph.ainvoke(initial_state, config=config)
    print(f"\n✅ 실행 성공!")
    print(f"마지막 메시지: {result['messages'][-1].content[:100]}...")
    print(f"현재 단계: {result.get('current_phase', 'Unknown')}")
    print(f"감정 힌트: {result.get('emotion_hints', [])}")
    print(f"상황 힌트: {result.get('situation_hints', [])}")
except Exception as e:
    print(f"❌ 기본 대화 테스트 실패: {e}")
    import traceback
    traceback.print_exc()

=== 기본 대화 테스트 (ASYNC) ===
실행 전 상태:
- current_phase: greeting
- consent_for_reco: False
DEBUG: Routing to END - phase: greeting, consent: False
예상 라우팅: __end__
=== NEW CONVERSATION_NODE STARTED ===
State: ['messages', 'current_phase', 'emotion_hints', 'situation_hints']
=== NEW CONVERSATION_NODE STARTED ===
DEBUG: Phase=greeting, User message=Hello, I'm feeling stressed from work.
DEBUG: Raw extracted hints - emotions: ['stressed'], situations: ['work stress']
DEBUG: Final merged hints - emotions: ['stressed'], situations: ['work stress']
DEBUG: Auto-triggered profile update due to hints
DEBUG: Memory analysis={'update_needed': True, 'update_type': 'profile', 'reason': "Emotion hints (['stressed']) or situation hints (['work stress']) collected"}
DEBUG: Response=Hey there! I'm here to chat and maybe help you dis...
DEBUG: Current emotion hints in state: []
DEBUG: Current situation hints in state: []
DEBUG: Final emotion hints: ['stressed']
DEBUG: Final situation hints: ['work stress']
D

In [7]:
# 스트리밍으로 각 노드 실행 과정 확인 (ASYNC)
print("=== 스트리밍으로 노드 실행 과정 확인 (ASYNC) ===")

stream_state = {
    "messages": [HumanMessage(content="Hello, I'm feeling stressed from work.")],
    "emotion_hints": [],
    "situation_hints": [],
    "current_phase": "greeting"
}

try:
    # ASYNC 스트리밍 사용
    async for step in graph.astream(stream_state, config=config):
        print(f"스텝: {step}")
        print("---")
except Exception as e:
    print(f"❌ 스트리밍 테스트 실패: {e}")
    import traceback
    traceback.print_exc()

=== 스트리밍으로 노드 실행 과정 확인 (ASYNC) ===
=== NEW CONVERSATION_NODE STARTED ===
State: ['messages', 'current_phase', 'emotion_hints', 'situation_hints']
=== NEW CONVERSATION_NODE STARTED ===
DEBUG: Phase=greeting, User message=Hello, I'm feeling stressed from work.
DEBUG: Raw extracted hints - emotions: ['stressed'], situations: ['work stress']
DEBUG: Final merged hints - emotions: ['stressed'], situations: ['work stress']
DEBUG: Auto-triggered profile update due to hints
DEBUG: Memory analysis={'update_needed': True, 'update_type': 'profile', 'reason': "Emotion hints (['stressed']) or situation hints (['work stress']) collected"}
DEBUG: Response=Hey there! I'm here to chat and maybe help you dis...
DEBUG: Current emotion hints in state: []
DEBUG: Current situation hints in state: []
DEBUG: Final emotion hints: ['stressed']
DEBUG: Final situation hints: ['work stress']
DEBUG: All state updates: ['emotion_hints', 'situation_hints']
DEBUG: About to update profile memory
DEBUG: Recent messages f

In [8]:
# Art Curation 트리거 조건 테스트 (ASYNC)
print("=== Art Curation 트리거 조건 테스트 (ASYNC) ===")

# ready_for_curation 상태로 설정
curation_ready_state = {
    "messages": [
        HumanMessage(content="I'm really stressed and overwhelmed. Can you recommend some art for me?"),
    ],
    "emotion_hints": ["stressed", "overwhelmed", "anxious"],
    "situation_hints": ["work pressure", "academic stress"],
    "current_phase": "ready_for_curation",
    "consent_for_reco": True
}

print("큐레이션 준비 상태:")
print(f"- current_phase: {curation_ready_state.get('current_phase')}")
print(f"- consent_for_reco: {curation_ready_state.get('consent_for_reco')}")

# should_curate_art로 라우팅 확인
routing_result = should_curate_art(curation_ready_state)
print(f"예상 라우팅: {routing_result}")

print("\n큐레이션 준비 상태로 스트리밍 테스트...")
try:
    async for step in graph.astream(curation_ready_state, config=config):
        print(f"스텝: {step}")
        print("---")
except Exception as e:
    print(f"❌ 큐레이션 트리거 테스트 실패: {e}")
    import traceback
    traceback.print_exc()

=== Art Curation 트리거 조건 테스트 (ASYNC) ===
큐레이션 준비 상태:
- current_phase: ready_for_curation
- consent_for_reco: True
DEBUG: Routing to art curation - phase: ready_for_curation, consent: True
예상 라우팅: art_curation_node

큐레이션 준비 상태로 스트리밍 테스트...
=== NEW CONVERSATION_NODE STARTED ===
State: ['messages', 'current_phase', 'emotion_hints', 'situation_hints', 'consent_for_reco']
=== NEW CONVERSATION_NODE STARTED ===
DEBUG: Phase=greeting, User message=I'm really stressed and overwhelmed. Can you recommend some art for me?
DEBUG: Raw extracted hints - emotions: ['stressed', 'overwhelmed'], situations: []
DEBUG: Final merged hints - emotions: ['stressed', 'overwhelmed', 'anxious'], situations: ['work pressure', 'academic stress']
DEBUG: Auto-triggered profile update due to hints
DEBUG: Memory analysis={'update_needed': True, 'update_type': 'profile', 'reason': "Emotion hints (['stressed', 'overwhelmed', 'anxious']) or situation hints (['work pressure', 'academic stress']) collected"}
DEBUG: Response=

INFO:art_curation_engine.core.langchain_rag_system:Setting up LangChain RAG components...
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2


🔄 Loading LangChain RAG system...


INFO:art_curation_engine.core.langchain_rag_system:✅ Embeddings model loaded: sentence-transformers/all-MiniLM-L6-v2
INFO:art_curation_engine.core.langchain_rag_system:✅ Text splitter configured
INFO:faiss.loader:Loading faiss.
INFO:faiss.loader:Successfully loaded faiss.
INFO:art_curation_engine.core.langchain_rag_system:✅ Vector store loaded from langchain_vectorstore
INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: mps
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2


Loaded existing vector store
✅ LangChain RAG system loaded successfully
✅ LLM setup complete (OpenAI)
📚 Extracting subject vocabulary from metadata...
✅ Extracted 353 unique subject terms
📊 Top 10 subjects: [('portraits', 261), ('portrait', 197), ('portraits: male subject', 103), ('man', 88), ('fashion', 85), ('portraits: female subject', 85), ('people', 76), ('woman', 63), ('century of progress', 60), ("world's fairs", 60)]
✅ Loaded 10 concept categories
✅ LLM setup complete for dynamic generation


...

✅ Model loaded successfully from local cache
✅ FAISS index loaded with 298 embeddings
📦 Dynamic generation cache hit in 0.000s
🎨 Generated 3 CLIP prompts:
   1. A serene landscape with a girl sitting by a calm body of wat...
   2. A portrait of a family enjoying a sunny day in a park, with ...
   3. An abstract representation of anxiety, using swirling blue a...
   🔍 Searching prompt 1/3: A serene landscape with a girl sitting by a calm body of wat...
   🔍 Searching prompt 2/3: A portrait of a family enjoying a sunny day in a park, with ...
   🔍 Searching prompt 3/3: An abstract representation of anxiety, using swirling blue a...


INFO:art_curation_engine.core.step6_llm_reranking:Starting Step 6 reranking: 3 → 8 candidates
INFO:art_curation_engine.core.step6_llm_reranking:Phase 1: Scoring candidates with LLM...


Traceback (most recent call last):
  File "/Users/ijunhyeong/Desktop/arti_llm/studio/emotional_art_graph.py", line 1511, in art_curation_node
    final_recs = await loop.run_in_executor(
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/envs/arti_chatbot/lib/python3.11/concurrent/futures/thread.py", line 58, in run
    result = self.fn(*self.args, **self.kwargs)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/ijunhyeong/Desktop/arti_llm/studio/art_curation_engine/core/step6_llm_reranking.py", line 259, in rerank_candidates
    all_scores = self._score_all_candidates(rag_brief, candidates, brief_hash)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/ijunhyeong/Desktop/arti_llm/studio/art_curation_engine/core/step6_llm_reranking.py", line 306, in _score_all_candidates
    artwork_id = candidate.get('artwork_id', candidate.get('id', ''))
                 ^^^^^^^^^^^^^
AttributeError: 'str' object has no attribut

✅ A2 complete: 150 candidates from 239 total matches
🔗 Merging A1 and A2 results (target: 150)
✅ Merge complete: 150 final candidates
📦 Dynamic generation cache hit in 0.000s
🔑 Generated 10 open keywords: ['lake', 'girl', 'mountains', 'water', 'landscape', 'trees', 'blue', 'portraits', 'hat', 'ocean']
📦 Dynamic generation cache hit in 0.000s
🎨 Generated 3 CLIP prompts:
   1. A serene landscape with a girl sitting by a calm body of wat...
   2. A portrait of a family enjoying a sunny day in a park, with ...
   3. An abstract representation of anxiety, using swirling blue a...

📊 Stage A Results (balanced):
   Final candidates: 150
   A1 metadata hits: 200
   A2 CLIP hits: 150
   Generated keywords: 10
   CLIP prompts: 3

🎯 Generating session brief with dynamic LangChain RAG...
   Situation: general emotional support and mood enhancement
   Emotions: ['overwhelmed', 'stressed', 'tired']
✅ Using cached result

🎭 Stage A: Candidate Collection (balanced mode)
📝 Situation: general emotional 

DEBUG: Starting Step 6 - LLM reranking
DEBUG: Art curation error: 'str' object has no attribute 'get'
DEBUG: Restored working directory to /Users/ijunhyeong/Desktop/arti_llm
스텝: {'art_curation_node': {'messages': [AIMessage(content='I encountered an issue with the art database. Would you like to continue our conversation, and I can try the recommendations again later?', additional_kwargs={}, response_metadata={}, id='2cecc9d3-6f8d-442e-9f2c-a16c7fc9af32')], 'current_phase': 'continuing'}}
---


In [9]:
# prepare_curation_input 함수 테스트
from emotional_art_graph import prepare_curation_input

print("=== prepare_curation_input 함수 테스트 ===")

# 가짜 메모리 데이터
fake_memories = {
    'user_profile': {
        'current_situation': 'academic stress from research work',
        'emotional_context': 'feeling overwhelmed',
        'emotions': ['stressed', 'anxious', 'tired']
    },
    'art_preferences': {
        'preferred_styles': ['minimalist', 'abstract'],
        'effective_colors': ['blue', 'green']
    }
}

# 가짜 state 데이터 (EmotionalArtState와 유사하게)
class FakeState:
    def __init__(self, data):
        self.emotion_hints = data.get('emotion_hints', [])
        self.situation_hints = data.get('situation_hints', [])
    
    def get(self, key, default=None):
        return getattr(self, key, default)

fake_state = FakeState({
    'emotion_hints': ['overwhelmed', 'stressed', 'tired'],
    'situation_hints': ['academic pressure', 'research deadlines']
})

try:
    curation_input = prepare_curation_input(fake_memories, fake_state)
    print(f"✅ 큐레이션 입력 생성 성공:")
    print(f"- situation: {curation_input.get('situation')}")
    print(f"- emotions: {curation_input.get('emotions')}")
    print(f"- preferences: {curation_input.get('preferences')}")
    print(f"- confidence_level: {curation_input.get('confidence_level')}")
except Exception as e:
    print(f"❌ prepare_curation_input 테스트 실패: {e}")
    import traceback
    traceback.print_exc()

=== prepare_curation_input 함수 테스트 ===
DEBUG: conversation_state type: <class '__main__.FakeState'>
DEBUG: conversation_state content: <__main__.FakeState object at 0x1745e0650>
✅ 큐레이션 입력 생성 성공:
- situation: academic stress from research work with underlying feeling overwhelmed
- emotions: ['overwhelmed', 'stressed', 'tired', 'anxious']
- preferences: {'liked_styles': ['minimalist', 'abstract'], 'avoided_themes': [], 'effective_colors': ['blue', 'green'], 'preferred_periods': []}
- confidence_level: 1.0


In [10]:
# Art Curation Engine 직접 테스트
print("=== Art Curation Engine 직접 테스트 ===")

# art_curation_engine 디렉토리로 변경
import os
original_cwd = os.getcwd()
art_engine_path = '/Users/ijunhyeong/Desktop/arti_llm/art_curation_engine'

if os.path.exists(art_engine_path):
    os.chdir(art_engine_path)
    print(f"✅ 작업 디렉토리 변경: {art_engine_path}")
    
    try:
        from core import RAGSessionBrief
        
        rag_session = RAGSessionBrief()
        print("✅ RAGSessionBrief 로드 성공")
        
        # 간단한 테스트
        test_situation = "academic stress from research work"
        test_emotions = ["overwhelmed", "stressed", "tired"]
        
        print(f"테스트 입력: situation='{test_situation}', emotions={test_emotions}")
        
        brief = rag_session.generate_brief(test_situation, test_emotions)
        print(f"✅ RAG Brief 생성 성공: {type(brief)}")
        print(f"Brief keys: {list(brief.keys()) if isinstance(brief, dict) else 'Not a dict'}")
        
        if isinstance(brief, dict):
            print(f"Brief 샘플:")
            for key, value in list(brief.items())[:3]:  # 처음 3개만 출력
                print(f"  {key}: {str(value)[:100]}...")
        
    except Exception as e:
        print(f"❌ Art Curation Engine 테스트 실패: {e}")
        import traceback
        traceback.print_exc()
    
    finally:
        os.chdir(original_cwd)
        print(f"✅ 작업 디렉토리 복원: {original_cwd}")
else:
    print(f"❌ Art Curation Engine 경로 없음: {art_engine_path}")

INFO:core.langchain_rag_system:Setting up LangChain RAG components...
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2


=== Art Curation Engine 직접 테스트 ===
✅ 작업 디렉토리 변경: /Users/ijunhyeong/Desktop/arti_llm/art_curation_engine
🔄 Loading LangChain RAG system...


INFO:core.langchain_rag_system:✅ Embeddings model loaded: sentence-transformers/all-MiniLM-L6-v2
INFO:core.langchain_rag_system:✅ Text splitter configured
INFO:core.langchain_rag_system:✅ Vector store loaded from langchain_vectorstore


Loaded existing vector store
✅ LangChain RAG system loaded successfully
✅ LLM setup complete (OpenAI)
✅ RAGSessionBrief 로드 성공
테스트 입력: situation='academic stress from research work', emotions=['overwhelmed', 'stressed', 'tired']

🎯 Generating session brief with dynamic LangChain RAG...
   Situation: academic stress from research work
   Emotions: ['overwhelmed', 'stressed', 'tired']
✅ Using cached result
✅ RAG Brief 생성 성공: <class 'dict'>
Brief keys: ['brief', 'evidence_used', 'queries_used', 'user_input', 'metadata']
Brief 샘플:
  brief: {'situation_analysis': 'The user is experiencing academic stress from research work, leading to feel...
  evidence_used: [{'title': 'Color arousal and performance A comparis', 'year': 'Unknown', 'domain': 'Color Psycholog...
  queries_used: ['color psychology effects on stress reduction in academic environments', 'environmental design stra...
✅ 작업 디렉토리 복원: /Users/ijunhyeong/Desktop/arti_llm


In [11]:
# 전체 통합 테스트 - art_curation_node까지 실행 (ASYNC)
print("=== 전체 통합 테스트 (ASYNC) ===")

# Art curation을 트리거하는 상태
full_test_state = {
    "messages": [
        HumanMessage(content="I'm really struggling with stress from my research work. I feel overwhelmed and tired. Can you help me find some art that might make me feel better?")
    ],
    "emotion_hints": ["overwhelmed", "stressed", "tired"],
    "situation_hints": ["research work", "academic pressure"],
    "current_phase": "ready_for_curation",
    "consent_for_reco": True
}

print("아트 큐레이션 전체 파이프라인 테스트...")
try:
    result = await graph.ainvoke(full_test_state, config=config)
    print(f"✅ 전체 테스트 성공")
    print(f"마지막 메시지: {result['messages'][-1].content[:200]}...")
    print(f"현재 단계: {result.get('current_phase', 'Unknown')}")
except Exception as e:
    print(f"❌ 전체 테스트 실패: {e}")
    import traceback
    traceback.print_exc()

=== 전체 통합 테스트 (ASYNC) ===
아트 큐레이션 전체 파이프라인 테스트...
=== NEW CONVERSATION_NODE STARTED ===
State: ['messages', 'current_phase', 'emotion_hints', 'situation_hints', 'consent_for_reco']
=== NEW CONVERSATION_NODE STARTED ===
DEBUG: Phase=greeting, User message=I'm really struggling with stress from my research work. I feel overwhelmed and tired. Can you help me find some art that might make me feel better?
DEBUG: Raw extracted hints - emotions: ['stressed', 'overwhelmed', 'tired'], situations: ['work stress', 'mental health', 'need for relaxation']
DEBUG: Final merged hints - emotions: ['overwhelmed', 'stressed', 'tired'], situations: ['research work', 'academic pressure', 'work stress', 'mental health', 'need for relaxation']
DEBUG: Auto-triggered profile update due to hints
DEBUG: Memory analysis={'update_needed': True, 'update_type': 'profile', 'reason': "Emotion hints (['overwhelmed', 'stressed', 'tired']) or situation hints (['research work', 'academic pressure', 'work stress', 'mental 

INFO:art_curation_engine.core.langchain_rag_system:Setting up LangChain RAG components...
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2


DEBUG: Memory updated successfully: profile - 1 items
DEBUG: Routing to art curation - phase: ready_for_curation, consent: True
DEBUG: Starting art curation node
DEBUG: Loaded user memories: []
DEBUG: conversation_state type: <class 'dict'>
DEBUG: conversation_state content: {'messages': [HumanMessage(content="I'm really struggling with stress from my research work. I feel overwhelmed and tired. Can you help me find some art that might make me feel better?", additional_kwargs={}, response_metadata={}, id='88c8132d-e9df-458c-8393-b15c2d3c4c4d'), AIMessage(content="Hey there! I'm here to chat and maybe help you discover some amazing art along the way. How's your day going?", additional_kwargs={}, response_metadata={}, id='ca87ce68-fb9e-488a-aa92-2a480f9a1ad1')], 'current_phase': 'ready_for_curation', 'emotion_hints': ['overwhelmed', 'stressed', 'tired'], 'situation_hints': ['research work', 'academic pressure', 'work stress', 'mental health', 'need for relaxation'], 'consent_for_reco': T

INFO:art_curation_engine.core.langchain_rag_system:✅ Embeddings model loaded: sentence-transformers/all-MiniLM-L6-v2
INFO:art_curation_engine.core.langchain_rag_system:✅ Text splitter configured
INFO:art_curation_engine.core.langchain_rag_system:✅ Vector store loaded from langchain_vectorstore
INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: mps
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2


Loaded existing vector store
✅ LangChain RAG system loaded successfully
✅ LLM setup complete (OpenAI)
📚 Extracting subject vocabulary from metadata...
✅ Extracted 353 unique subject terms
📊 Top 10 subjects: [('portraits', 261), ('portrait', 197), ('portraits: male subject', 103), ('man', 88), ('fashion', 85), ('portraits: female subject', 85), ('people', 76), ('woman', 63), ('century of progress', 60), ("world's fairs", 60)]
✅ Loaded 10 concept categories
✅ LLM setup complete for dynamic generation


...

✅ Dynamic generation complete in 3.681s
   Keywords: 10
   Prompts: 3
🔑 Generated 10 open keywords: ['lake', 'girl', 'mountains', 'water', 'landscape', 'trees', 'blue', 'portraits', 'hat', 'ocean']
✅ A1 complete: 200 candidates (from 294 matches)
🎯 A2: CLIP text→image search (per_query: 140, cap: 150)
🔧 Loading CLIP model and FAISS index...
= Attempting to load model from local cache...


INFO:root:Instantiating model architecture: CLIP
INFO:root:Loading full pretrained weights from: /Users/ijunhyeong/.cache/huggingface/models--laion--CLIP-ViT-B-32-laion2B-s34B-b79K/snapshots/1a25a446712ba5ee05982a381eed697ef9b435cf/open_clip_model.safetensors
INFO:root:Final image preprocessing configuration set: {'size': (224, 224), 'mode': 'RGB', 'mean': (0.48145466, 0.4578275, 0.40821073), 'std': (0.26862954, 0.26130258, 0.27577711), 'interpolation': 'bicubic', 'resize_mode': 'shortest', 'fill_color': 0}
INFO:root:Model ViT-B-32 creation process complete.
INFO:art_curation_engine.core.step6_llm_reranking:Starting Step 6 reranking: 3 → 8 candidates
INFO:art_curation_engine.core.step6_llm_reranking:Phase 1: Scoring candidates with LLM...


Traceback (most recent call last):
  File "/Users/ijunhyeong/Desktop/arti_llm/studio/emotional_art_graph.py", line 1511, in art_curation_node
    final_recs = await loop.run_in_executor(
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/envs/arti_chatbot/lib/python3.11/concurrent/futures/thread.py", line 58, in run
    result = self.fn(*self.args, **self.kwargs)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/ijunhyeong/Desktop/arti_llm/studio/art_curation_engine/core/step6_llm_reranking.py", line 259, in rerank_candidates
    all_scores = self._score_all_candidates(rag_brief, candidates, brief_hash)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/ijunhyeong/Desktop/arti_llm/studio/art_curation_engine/core/step6_llm_reranking.py", line 306, in _score_all_candidates
    artwork_id = candidate.get('artwork_id', candidate.get('id', ''))
                 ^^^^^^^^^^^^^
AttributeError: 'str' object has no attribut

✅ Model loaded successfully from local cache
✅ FAISS index loaded with 298 embeddings
📦 Dynamic generation cache hit in 0.000s
🎨 Generated 3 CLIP prompts:
   1. A serene landscape with a girl sitting by a calm body of wat...
   2. A portrait of a family enjoying a sunny day in a park, weari...
   3. An abstract representation of tiredness, using swirling blue...
   🔍 Searching prompt 1/3: A serene landscape with a girl sitting by a calm body of wat...
   🔍 Searching prompt 2/3: A portrait of a family enjoying a sunny day in a park, weari...
   🔍 Searching prompt 3/3: An abstract representation of tiredness, using swirling blue...
✅ A2 complete: 150 candidates from 235 total matches
🔗 Merging A1 and A2 results (target: 150)
✅ Merge complete: 150 final candidates
📦 Dynamic generation cache hit in 0.000s
🔑 Generated 10 open keywords: ['lake', 'girl', 'mountains', 'water', 'landscape', 'trees', 'blue', 'portraits', 'hat', 'ocean']
📦 Dynamic generation cache hit in 0.000s
🎨 Generated 3 CLIP

DEBUG: Starting Step 6 - LLM reranking
DEBUG: Art curation error: 'str' object has no attribute 'get'
DEBUG: Restored working directory to /Users/ijunhyeong/Desktop/arti_llm
✅ 전체 테스트 성공
마지막 메시지: I encountered an issue with the art database. Would you like to continue our conversation, and I can try the recommendations again later?...
현재 단계: continuing
